In [1]:
'''

the model consists of autoenocer and classifier
there are 2 loss functions for those two parts
sounds suitable for noisy data and not enough data, 

Input Image -> [Encoder Layers] -> Latent Space -> [Decoder Layers] -> Reconstructed Image
                                 |
                                 -> Fully Connected Classification Head -> Predicted Class

for both loss functions BCE is used
the model use sigmoid in the last layer

input image size: 64 X 64

'''

'\n\nthe model consists of autoenocer and classifier\nthere are 2 loss functions for those two parts\nsounds suitable for noisy data and not enough data, \n\nInput Image -> [Encoder Layers] -> Latent Space -> [Decoder Layers] -> Reconstructed Image\n                                 |\n                                 -> Fully Connected Classification Head -> Predicted Class\n\nfor both loss functions BCE is used\nthe model use sigmoid in the last layer\n\ninput image size: 64 X 64\n\n'

In [1]:
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

from config import Config, Device
from datasets import BalancedMRIDataset
from models import autoencoder
from trainer import Trainer_AutoencoderClassification
from tester import Tester_AutoencoderClassification


In [2]:
device = Device.device
print(device)

cpu


In [3]:
data_path = os.path.join(os.getcwd(), "data")
labels_path = "train.csv"

batch_size = Config.batch_size
num_epochs = Config.num_epochs
learning_rate = Config.learning_rate
mean = Config.mean  # mean of the entire datasaet
std = Config.std  # std of the entire dataaset
image_size = 224

In [4]:
resclaed_mean = round(mean/255, 4)  # re-scale the actual mean
rescaled_std = round(std/255, 4)  # re-scale the actual std

train_transforms = transforms.Compose([
    transforms.RandomRotation(degrees=10),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

augment_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(10),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

test_transforms = transforms.Compose([
    # transforms.Lambda(lambda img: img.astype(np.float32)),
    transforms.ToTensor(),
    transforms.Resize((64, 64)),
    transforms.Normalize(mean=[resclaed_mean], std=[rescaled_std])
])

In [5]:

train_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='train',
    transform=train_transforms,
    augment_transform=augment_transforms,
    augment=True,
    max_slices=20
)

val_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='val',
    transform=test_transforms,
    max_slices=20
)

test_dataset = BalancedMRIDataset(
    data_path,
    labels_path,
    split='test',
    transform=test_transforms,
    max_slices=20
)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dl = DataLoader(val_dataset, batch_size=32)
test_dl = DataLoader(test_dataset, batch_size=32)

In [6]:
data_, label_ = next(iter(train_dl))
data_.size()

torch.Size([32, 20, 64, 64])

In [7]:
model = autoencoder.AutoencoderClassifier().to(device=device)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion_reconstruction = nn.BCEWithLogitsLoss()  # For reconstruction loss
criterion_classification = nn.BCEWithLogitsLoss()   # For classification loss

In [9]:
model_name = model.__class__.__name__
model_name

'AutoencoderClassifier'

In [ ]:
trainer = Trainer_AutoencoderClassification(
    model=model,
    criterion=[criterion_reconstruction , criterion_classification],
    optimizer=optimizer,
    train_dl=train_dl,
    val_dl=val_dl,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    device=device,
    num_epochs=1,
    patience=100,
    threshold=0.5,
    save_path=f"saved_models/{model_name}.pth"
)

# Start training
trainer.train()

In [12]:
model.load_state_dict(torch.load(f"saved_models/{model_name}_best.pth"))

<All keys matched successfully>

In [13]:
tester = Tester_AutoencoderClassification(
    model=model,
    criterion=[criterion_reconstruction , criterion_classification],
    test_dl=test_dl,
    test_dataset=test_dataset,
    device=device,
    threshold=0.5
)

tester.test()

Test Loss: -18769.9094, Test Accuracy: 0.8754
Precision: 0.0000, Recall: 0.0000, AUC: 0.5000, Avg Metric: 0.0000
Confusion Matrix:
[[548   0]
 [ 78   0]]


In [15]:
# !tensorboard --logdir=runs/training/{self.model.__class__.__name__}

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6006/ (Press CTRL+C to quit)
E0903 18:11:22.356342 6213545984 _internal.py:97] Error on request:
Traceback (most recent call last):
  File "/Users/anazarnia/data_science/myenv_1/lib/python3.9/site-packages/werkzeug/serving.py", line 363, in run_wsgi
    execute(self.server.app)
  File "/Users/anazarnia/data_science/myenv_1/lib/python3.9/site-packages/werkzeug/serving.py", line 324, in execute
    application_iter = app(environ, start_response)
  File "/Users/anazarnia/data_science/myenv_1/lib/python3.9/site-packages/tensorboard/backend/application.py", line 528, in __call__
    return self._app(environ, start_response)
  File "/Users/anazarnia/data_science/myenv_1/lib/python3.9/site-packages/tensorboard/backend/application.py", line 569, in wrapper
    return wsgi_app(environ, start_respon

In [14]:

# # Hyperparameters
# num_epochs = 50
# batch_size = 32
# learning_rate = 0.001

# # Create DataLoader (Assuming `train_images` and `train_labels` are loaded)
# transform = transforms.Compose([transforms.ToTensor()])
# train_dataset = BrainImageDataset(train_images, train_labels, transform=transform)
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# # Model, Loss, Optimizer
# model = AutoencoderClassifier()
# criterion_reconstruction = nn.BCELoss()  # For reconstruction loss
# criterion_classification = nn.BCELoss()   # For classification loss
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Training Loop
# for epoch in range(num_epochs):
#     for images, labels in train_loader:
#         # Flatten the images for the classifier
#         optimizer.zero_grad()
        
#         # Forward pass
#         reconstructed, outputs = model(images)
        
#         # Compute losses
#         loss_reconstruction = criterion_reconstruction(reconstructed, images)
#         loss_classification = criterion_classification(outputs.squeeze(), labels.float())
#         loss = loss_reconstruction + loss_classification
        
#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # Saving the model
# torch.save(model.state_dict(), "autoencoder_classifier.pth")